In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time, os
import pandas as pd
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model
from PIL import ImageFont, ImageDraw, Image
from google.protobuf.json_format import MessageToDict

In [27]:
labels = pd.read_csv('./dataset_1/label.csv')
labels = labels.values

In [30]:
actions = labels
# LSTM Window Size
seq_length = 30

VIDEO_FILES = []
dir_path = './dataset_1/viedo'
for (root, directories, files) in os.walk(dir_path):
    for file in files:
        VIDEO_FILES.append(os.path.join(root, file))

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    model_complexity=0,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

os.makedirs('dataset', exist_ok=True)
zero = np.zeros(63) 

for idx, file in enumerate(VIDEO_FILES):
    cap = cv2.VideoCapture(file)
    action = actions[idx][0]
    data = []

    while cap.isOpened():

        ret, img = cap.read()
        if not ret:
            break
            
#         img = cv2.resize(img, dsize=(600,600))
        img = cv2.flip(img, 1)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = hands.process(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        if result.multi_hand_landmarks:
            data_arr = []
            for res in result.multi_hand_landmarks:
                joint = np.zeros((21, 3))
                for j, lm in enumerate(res.landmark):
                    joint[j] = [lm.x, lm.y, lm.z]

                mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)
            
                data_arr.extend(joint.flatten())
               
            if len(data_arr) == 63:
                handedness_dict = MessageToDict(result.multi_handedness[0])
                if handedness_dict['classification'][0]['label'] == 'Right':
                    data_arr = np.concatenate((zero, data_arr))
                else:
                    data_arr = np.concatenate((data_arr, zero))
            elif len(data_arr) > 126:
                continue
            data_arr = np.append(data_arr, idx)
            data.append(data_arr)
            
        cv2.imshow('img', img)
        if cv2.waitKey(1) == ord('q'):
            break
    
    data = np.array(data)

    # Create sequence data
    try:
        full_seq_data = []
        for seq in range(len(data) - seq_length):
            full_seq_data.append(data[seq:seq + seq_length])
        full_seq_data = np.array(full_seq_data)

        if idx == 0:
            full_data = full_seq_data
            continue

        full_data = np.concatenate((full_data, full_seq_data))
        print(action,idx, full_data.shape)
        
    except:
        print('ERROR!!', action, idx)
        pass

cv2.destroyAllWindows()
cap.release()

np.save(os.path.join('dataset/fulldata'), full_data)

개 1 (55, 30, 127)
공원 2 (84, 30, 127)
금요일 3 (121, 30, 127)
내년 4 (142, 30, 127)
ERROR!! 내일 5
냄새나다 6 (160, 30, 127)
누나 7 (180, 30, 127)
동생 8 (203, 30, 127)
목요일 9 (226, 30, 127)
아래 10 (260, 30, 127)
바다 11 (295, 30, 127)
배고프다 12 (332, 30, 127)
병원 13 (355, 30, 127)
불 14 (393, 30, 127)
산 15 (423, 30, 127)
삼키다 16 (443, 30, 127)
선생님 17 (470, 30, 127)
수요일 18 (485, 30, 127)
아빠 19 (511, 30, 127)
아파트 20 (546, 30, 127)
앞 21 (565, 30, 127)
어제 22 (576, 30, 127)
어지러움 23 (595, 30, 127)
언니 24 (618, 30, 127)
엄마 25 (630, 30, 127)
오늘 26 (650, 30, 127)
오른쪽 27 (678, 30, 127)
오빠 28 (696, 30, 127)
올해 29 (730, 30, 127)
왼쪽 30 (764, 30, 127)
월요일 31 (776, 30, 127)
위에 32 (810, 30, 127)
음식물 33 (852, 30, 127)
일요일 34 (894, 30, 127)
자동차 35 (934, 30, 127)
작년 36 (974, 30, 127)
집 37 (990, 30, 127)
택시 38 (1026, 30, 127)
토요일 39 (1058, 30, 127)
학교 40 (1080, 30, 127)
형 41 (1090, 30, 127)
화요일 42 (1108, 30, 127)
화장실 43 (1122, 30, 127)
0 44 (1154, 30, 127)
1 45 (1193, 30, 127)
2 46 (1230, 30, 127)
3 47 (1264, 30, 127)
4 48 (1303,

집 359 (14626, 30, 127)
친구 360 (14669, 30, 127)
택시 361 (14714, 30, 127)
토요일 362 (14756, 30, 127)
학교 363 (14797, 30, 127)
형 364 (14819, 30, 127)
화요일 365 (14840, 30, 127)
화장실 366 (14861, 30, 127)
6 367 (14892, 30, 127)
7 368 (14922, 30, 127)
0 369 (14930, 30, 127)
1 370 (14941, 30, 127)
2 371 (14959, 30, 127)
3 372 (14979, 30, 127)
4 373 (14995, 30, 127)
5 374 (15013, 30, 127)
6 375 (15028, 30, 127)
7 376 (15045, 30, 127)
8 377 (15061, 30, 127)
9 378 (15074, 30, 127)
10 379 (15094, 30, 127)
가렵다 380 (15106, 30, 127)
개 381 (15139, 30, 127)


In [32]:
data1 = np.load('./dataset/fulldata.npy')
data1.shape

(15139, 30, 127)